In [1]:
from library.Mag import *
from SimPEG import PF, Utils, Mesh
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [8]:
# Create a survey
fileName = './DIGHEM_Mag_floor10nt_25m.obs'
xyzd = np.genfromtxt(fileName, skip_header=3)
B = np.r_[60308, 83.8, 25.4]
survey = Mag.createMagSurvey(xyzd, B)
cntr = np.mean(xyzd[:, :2], axis=0)

prisms = []
# Create a synthetic block model and place it at the center of the survey
prisms.append( Simulator.definePrism())
prisms[0].x0, prisms[0].y0, prisms[0].z0 = cntr[0]+100, cntr[1]-100, xyzd[:, 2].min() - 50
prisms[0].dx, prisms[0].dy, prisms[0].dz = 200, 200 ,100
prisms[0].pdec = -30

prisms.append( Simulator.definePrism())
prisms[1].x0, prisms[1].y0, prisms[1].z0 = cntr[0]-1500, cntr[1], xyzd[:, 2].min()-600
prisms[1].dx, prisms[1].dy, prisms[1].dz = 2000, 2500 ,1000
prisms[1].pdec, prisms[1].pinc = -45, 20

prisms.append( Simulator.definePrism())
prisms[2].x0, prisms[2].y0, prisms[2].z0 = cntr[0]+600, cntr[1]-200, xyzd[:, 2].min()-200
prisms[2].dx, prisms[2].dy, prisms[2].dz = 4000, 100 ,500
prisms[2].pdec, prisms[2].pinc = 60, -30

for prism in prisms:
    prob = Mag.problem(prism=prism, survey=survey)
    survey.dobs += prob.fields()[0]

def Prism(View_dip,View_azm,View_lim):

    Simulator.plotObj3D(prisms, survey, View_dip,View_azm,View_lim, colors=['w','r','g'])
    
    
out = widgets.interactive(Prism,
                          View_dip=widgets.FloatSlider(min=0, max=90, step=1, value=75, continuous_update=False),
                          View_azm=widgets.FloatSlider(min=0, max=360, step=1, value=270, continuous_update=False),
                          View_lim=widgets.FloatSlider(min=1, max=2000, step=1, value=800, continuous_update=False),
                          )
# View prism
display(out)

In [9]:
param = Simulator.ViewMagSurvey2D(survey)
display(param)

In [10]:
gridCC, dMinCurv = Simulator.minCurvatureInterp(
    xyzd[:,:2], survey.dobs,
    vectorX=None, vectorY=None, vectorZ=None, gridSize=10,
    tol=1e-5, iterMax=None, method='spline'
)

X = gridCC[:,0].reshape(dMinCurv.shape, order='F')
Y = gridCC[:,1].reshape(dMinCurv.shape, order='F')

In [11]:
def dataViewer(X,Y,dataGrid):
      
    def plotData(azdeg,altdeg,alpha,alphaHS,vScale,contour):
        fig = plt.figure(figsize=(8,6))
        axs= plt.subplot()

        # Read the data
#         vmin, vmax = -25, 150
        cmap = plt.cm.gist_earth

        # Add shading
        im, CS = Simulator.plotDataHillside(X, Y,dataGrid ,
                                  axs=axs, cmap='RdBu_r', 
                                  clabel=False, contour=contour,
                                  alpha=alpha, alphaHS=alphaHS, ve=vScale, azdeg=azdeg, altdeg=altdeg)

        # Add points at the survey locations
        plt.scatter(xyzd[:,0], xyzd[:,1], s=2, c='k')

        # Set limits and labels
#         axs.set_xlim([xmin,xmax])
#         axs.set_ylim([ymin,ymax])
        # axs.set_xticklabels([556750,558000, 559250], size=14)
        # axs.set_xticks([556750,558000, 559250])
        # axs.set_yticklabels([7133000,7134000, 7135000], size=14, rotation=90, va='center')
        plt.colorbar(im)
        # axs.set_yticks([7133000,7134000, 7135000])
        axs.set_xlabel("Easting (m)", size=14)
        axs.set_ylabel("Northing (m)", size=14)
        axs.grid('on', color='k', linestyle='--')
        plt.show()
        # Add colorbar
        # pos = axs.get_position() # Get the position of previous axes
        # axbar = plt.axes([pos.x0+.25, pos.y0+0.2,  pos.width*0.75, pos.height*.5]) # Create a new axes and reshape
        # cbar = plt.colorbar(im, orientation='vertical',format='%.3f') # Add a colorbar using the color definition of previous
        # cbar.set_ticks(np.linspace(-25,300,6)) # Change the tick position
        # cbar.set_ticklabels(np.round(np.linspace(-25,300,6))) # Change the tick labels
        # axbar.text(pos.x0+1., pos.y0+.75,'SI', size=14) # Add units and move it above
        # axbar.axis('off') # Only keep the colorbar, remove the rest

    out = widgets.interactive(plotData,
                              azdeg=widgets.FloatSlider(min=0, max=360, step=5, value=0, continuous_update=False),
                              altdeg=widgets.FloatSlider(min=0, max=90, step=5, value=45, continuous_update=False),
                              alpha=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.2, continuous_update=False),
                              alphaHS=widgets.FloatSlider(min=0, max=1, step=0.1, value=1.0, continuous_update=False),
                              vScale=widgets.FloatSlider(min=1, max=1000, step=1., value=1.0, continuous_update=False),
                              contour=widgets.FloatSlider(min=0, max=20, step=1, value=10, continuous_update=False))
    return out

view = dataViewer(X, Y, dMinCurv)
display(view)